<a href="https://colab.research.google.com/github/swarmee/Financial-Intelligence-1/blob/master/FI_Comp_2020_example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FI Comp 2020 - Example Objecive Notebook 
This notebook provides and example of an objective and provides an basic python implemenation of accessing the API.

To minimise setup steps and end user requirements this notebook can be opened directly in google colabs. 


# Input Credentials

In [12]:
#@title Input Credentials 
#@markdown First things first input your credentials 

apiUrl   = 'https://api.swarmee.net'  #@param {type: "string"}
userName = 'example'  #@param {type: "string"}
password = 'form'  #@param {type: "string"}

#@markdown Make sure you you run this cell. 

#@markdown ---

print('username and password loaded')

username and password loaded


# Getting Some Sample Data

In [31]:
## Pull single Report to Allow Familiarisation with Data 
import requests
import json

response = requests.get(apiUrl + '/v1/dataset/transaction/?batchSize=3', auth=(userName, password))
if response.status_code != 200:
  print('something went your with your request')
  print(response.text)
else:
  headers = response.headers
  data    = response.json()
  print(json.dumps(data, indent=2))

{
  "transaction": {
    "content": [
      {
        "transactio": {
          "links": [
            {
              "ref": "self",
              "href": "https://api.swarmee.net/v1/dataset/transaction/jUsTj3ABB9N8a0VbGkW2"
            }
          ],
          "content": {
            "role": [
              {
                "roleType": "orderingCustomer",
                "party": [
                  {
                    "account": [
                      {
                        "network": "swift",
                        "branchName": "BANCA MONTE DEI PASCHI DI SIENA S.P.A.",
                        "country": "IT",
                        "suburb": "Monteleone D'Orvieto",
                        "branchId": "PASCITM1TR7",
                        "number": "15-003-1382"
                      }
                    ],
                    "identification": [
                      {
                        "type": "birthDate",
                        "identifer": "1961-09-12"
      

# Get all date for a Day

In [37]:
## Pull Each request and extract relevant details into List
import requests

consolidatedResponseData = []

def process_response(response):
  global consolidatedResponseData
  if response.status_code != 200:
    print('something went your with your request')
    print(response.text)
    return None
  else:
    nextPart = response.headers.get('X-swarmee-nextPartId', None)
    data     = response.json()
    consolidatedResponseData = consolidatedResponseData + [
      {'reportType' :  x['transactio']['content']['report']['reportType'] ,
       'amount'     :  x['transactio']['content']['transaction']['amount'], 
       'direction'  :  x['transactio']['content']['transaction']['direction'] } for x in data['transaction']['content'] ]
    print('Batch Obtained')
    return nextPart

initialResponse = requests.get(apiUrl + '/v1/dataset/transaction/?batchSize=1000', auth=(userName, password))
nextPart        = process_response(initialResponse)

while nextPart != None:
   subsequentResponse = requests.get(nextPart, auth=(userName, password))
   nextPart= process_response(subsequentResponse)

print(str(len(consolidatedResponseData)) + ' reports downloaded')

Batch Obtained
Batch Obtained
Batch Obtained
Batch Obtained
Batch Obtained
Batch Obtained
4003 reports downloaded


# Convert to Dataframe

In [42]:
## convert to Dataframe
import pandas as pd

df = pd.DataFrame(consolidatedResponseData)
df['amount'] = df['amount'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4003 entries, 0 to 4002
Data columns (total 3 columns):
reportType    4003 non-null object
amount        4003 non-null float64
direction     4003 non-null object
dtypes: float64(1), object(2)
memory usage: 93.9+ KB


# Produce Visualisation

In [50]:
## create visulisation
import plotly.express as px

fig = px.scatter_matrix(df) 
fig =  px.scatter(df, x="direction", y="amount", color="reportType",     marginal_y="box", log_y=True)
fig.show()